In [ ]:
# === colab configuration ===
# Skip this cell if running demo file locally!

from google.colab import drive
import sys

# setting paths
repository_path = '/content/time-series-forecasting-with-transformers/'
datasets_path = repository_path + 'datasets/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/francescobaraldi/time-series-forecasting-with-transformers
# ! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main'
! git checkout $branch
! git pull origin $branch

! pip install yfinance

executing_on_colab = True

In [ ]:
import torch
import joblib
import yfinance as yf

from model import StockTransformerDecoder, StockTransformer, StockLSTM
from inference import inference_transformer_decoder, inference_transformer, inference_lstm

try:
    executing_on_colab
except NameError:
    executing_on_colab = False

In [ ]:
if not executing_on_colab:
    best_model_path = "best_models/"
    scaler_path = "weights/scaler_split_80.gz"
    inference_path = "inference_results/"
else:
    best_model_path = "/content/drive/My Drive/time-series-forecasting-with-transformers/best_models/"
    scaler_path = "/content/drive/My Drive/time-series-forecasting-with-transformers/weights/scaler_split_70.gz"
    inference_path = "/content/drive/My Drive/time-series-forecasting-with-transformers/inference_results/"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
window_len = 90
forecast_len = 30

transformer_decoder_args = {
    'window_len': window_len,
    'num_layers': 1,
    'input_size': 1,
    'output_size': 1,
    'd_model': 128,
    'num_heads': 8,
    'feedforward_dim': 256,
    'dropout': 0,
    'positional_encoding': 'sinusoidal',
}
transformer_args = {
    'window_len': window_len,
    'target_len': forecast_len,
    'num_encoder_layers': 1,
    'num_decoder_layers': 1,
    'input_size': 1,
    'output_size': 1,
    'd_model': 128,
    'num_heads': 8,
    'feedforward_dim': 256,
    'dropout': 0,
    'positional_encoding': 'sinusoidal',
}
lstm_args = {
    'input_size': 1,
    'hidden_dim': 64,
    'output_size': 1,
    'num_layers': 2,
    'dropout': 0,
}

transformer_decoder = StockTransformerDecoder(**transformer_decoder_args)
transformer = StockTransformer(**transformer_args)
lstm = StockLSTM(**lstm_args)
transformer_decoder.load_state_dict(torch.load(best_model_path + "best_transformer_decoder.pth", map_location=torch.device(device)))
transformer.load_state_dict(torch.load(best_model_path + "best_transformer.pth", map_location=torch.device(device)))
lstm.load_state_dict(torch.load(best_model_path + "best_lstm.pth", map_location=torch.device(device)))

scaler = joblib.load(scaler_path)
gspc = yf.Ticker("^GSPC")
gspc = gspc.history(period="1y")

## Forecasting with transformer decoder model

In [ ]:
print("Forecasting the SP500 index closing price for the next 30 days with transformer decoder model...")
    
data = gspc[['Close']].iloc[-window_len:].to_numpy()
scaler = joblib.load(scaler_path)
data_scaled = scaler.transform(data)
src = torch.from_numpy(data_scaled).float().unsqueeze(0)
inference_transformer_decoder(device=device, model=transformer_decoder, src=src, forecast_len=forecast_len, scaler=scaler,
                              save_path=inference_path + "prediction_transformer_decoder.png")

print(f"The prediction of the transformer decoder model has been saved correctly in folder {inference_path}")

## Forecasting with transformer model

In [ ]:
print("Forecasting the SP500 index closing price for the next 30 days with transformer model...")

n = window_len + forecast_len - 1
data = gspc[['Close']].iloc[-n:].to_numpy()
scaler = joblib.load(scaler_path)
data_scaled = scaler.transform(data)
input = torch.from_numpy(data_scaled).float().unsqueeze(0)
inference_transformer(device=device, model=transformer, input=input, window_len=window_len, forecast_len=forecast_len,
                      scaler=scaler, save_path=inference_path + "prediction_transformer.png")

print(f"The prediction of the transformer model has been saved correctly in folder {inference_path}")

## Forecasting with LSTM model

In [ ]:
print("Forecasting the SP500 index closing price for the next 30 days with lstm model...")
    
data = gspc[['Close']].iloc[-window_len:].to_numpy()
scaler = joblib.load(scaler_path)
data_scaled = scaler.transform(data)
src = torch.from_numpy(data_scaled).float().unsqueeze(0)
inference_lstm(device=device, model=lstm, src=src, forecast_len=forecast_len, scaler=scaler,
               save_path=inference_path + "prediction_lstm.png")

print(f"The prediction of the lstm model has been saved correctly in folder {inference_path}")